In [1]:
!pip install datasets -qq

In [2]:
!pip install transformers -qq

In [3]:
!pip install accelerate -qq

In [4]:
!pip install fuzzywuzzy -qq

In [5]:
!pip install python-Levenshtein -qq

In [6]:
import random
from typing import Dict, List, Union

In [7]:
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from transformers import pipeline

In [8]:
# from typing import Union, List, Dict

# def prepare_message_for_llm(text: Union[str, List[str]], categories: Dict[str, str]) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:
#     if len(categories) < 2:
#         raise ValueError(f'Необходимо предоставить хотя бы две категории. Получено {len(categories)} категорий.')

#     categories_list = sorted(list(categories.keys()))  # Сортируем ключи словаря для стабильности
#     categories_as_string = ', '.join(categories_list[:-1]) + ' и ' + categories_list[-1]

#     # Примеры для обучения модели
#     examples = '\n'.join([f'Текст: {" ".join(categories[key].split())}\nВаш ответ: {key}' for key in categories_list])

#     if isinstance(text, str):
#         prompt = (
#             f'Вы являетесь классификатором тем текстов. Вам будет предоставлен текст, и ваша задача — '
#             f'определить, какая тема из предложенного списка наиболее подходит этому тексту. В ответе '
#             f'укажите только название темы, никаких других комментариев не требуется.\n\n'
#             f'Предложенный список тем: {categories_as_string}.\n\n'
#             f'Ниже приведены примеры, чтобы вы могли понять, как работать с заданием:\n{examples}\n\n'
#             f'Теперь ваш текст:\n{" ".join(text.split())}\nВаш ответ: '
#         )

#         messages = [
#             {
#                 'role': 'system',
#                 'content': 'Вы - классификатор тем текстов.'
#             },
#             {
#                 'role': 'user',
#                 'content': prompt
#             }
#         ]
#     else:
#         messages = []
#         for it in text:
#             prompt = (
#                 f'Вы являетесь классификатором тем текстов. Вам будет предоставлен текст, и ваша задача — '
#                 f'определить, какая тема из предложенного списка наиболее подходит этому тексту. В ответе '
#                 f'укажите только название темы, никаких других комментариев не требуется.\n\n'
#                 f'Предложенный список тем: {categories_as_string}.\n\n'
#                 f'Ниже приведены примеры, чтобы вы могли понять, как работать с заданием:\n{examples}\n\n'
#                 f'Теперь ваш текст:\n{" ".join(it.split())}\nВаш ответ: '
#             )

#             messages.append([
#                 {
#                     'role': 'system',
#                     'content': 'Вы - классификатор тем текстов.'
#                 },
#                 {
#                     'role': 'user',
#                     'content': prompt
#                 }
#             ])

#     return {'message_for_llm': messages}

In [9]:
from typing import Union, List, Dict

def prepare_message_for_llm(text: Union[str, List[str]], categories: Dict[str, str], examples_count=3) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:
    if len(categories) < 2:
        raise ValueError(f'Необходимо предоставить хотя бы две категории. Получено {len(categories)} категорий.')

    categories_list = sorted(list(categories.keys()))  # Сортируем ключи словаря для стабильности
    categories_as_string = ', '.join(categories_list[:-1]) + ' и ' + categories_list[-1]

    # Выбираем случайные примеры для few-shot подхода
    import random
    random_examples = random.sample(list(categories.items()), min(examples_count, len(categories)))

    # Формируем примеры для обучения модели
    examples = '\n'.join([f'Текст: {" ".join(example_value.split())}\nВаш ответ: {example_key}' for example_key, example_value in random_examples])

    if isinstance(text, str):
        prompt = (
            f'Вы являетесь классификатором тем текстов. Вам будет предоставлен текст, и ваша задача — '
            f'определить, какая тема из предложенного списка наиболее подходит этому тексту. В ответе '
            f'укажите только название темы, никаких других комментариев не требуется.\n\n'
            f'Предложенный список тем: {categories_as_string}.\n\n'
            f'Ниже приведены примеры, чтобы вы могли понять, как работать с заданием:\n{examples}\n\n'
            f'Теперь ваш текст:\n{" ".join(text.split())}\nВаш ответ: '
        )

        messages = [
            {
                'role': 'system',
                'content': 'Вы - классификатор тем текстов.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    else:
        messages = []
        for it in text:
            prompt = (
                f'Вы являетесь классификатором тем текстов. Вам будет предоставлен текст, и ваша задача — '
                f'определить, какая тема из предложенного списка наиболее подходит этому тексту. В ответе '
                f'укажите только название темы, никаких других комментариев не требуется.\n\n'
                f'Предложенный список тем: {categories_as_string}.\n\n'
                f'Ниже приведены примеры, чтобы вы могли понять, как работать с заданием:\n{examples}\n\n'
                f'Теперь ваш текст:\n{" ".join(it.split())}\nВаш ответ: '
            )

            messages.append([
                {
                    'role': 'system',
                    'content': 'Вы - классификатор тем текстов.'
                },
                {
                    'role': 'user',
                    'content': prompt
                }
            ])

    return {'message_for_llm': messages}

# Pipeline
## Загрузим модель

In [10]:
llm_pipeline = pipeline(model='Qwen/Qwen2-7B-Instruct', device_map='auto', torch_dtype='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

 ## Загрузим датасет

In [11]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

## Выделим категории

In [12]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Categories for classification are: {list_of_categories}')

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [13]:
print(validation_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 99
})


## Выделим случайные примеры

In [14]:
examples_by_categories = dict()
for current_category in list_of_categories:
    examples_by_categories[current_category] = random.choice(
        train_set.filter(lambda it: it['category'] == current_category)['text']
    )
    print(f'Category: {current_category}\n')
    print(f'Random text: {examples_by_categories[current_category]}\n\n')

Category: entertainment

Random text: Это культурное событие также является частью кампании мэрии Бухареста по превращению столицы Румынии в яркий творческий мегаполис.


Category: geography

Random text: Метеорологическая служба Исландии также не сообщала о каких-либо признаках сейсмической активности в районе Геклы за прошедшие 48 часов.


Category: health

Random text: В понедельник ученые из Медицинской школы Стэнфордского университета объявили об изобретении нового диагностического инструмента, который может сортировать клетки по их типу; это маленький чип, который можно напечатать, используя стандартный струйный принтер примерно за 1 цент США.


Category: politics

Random text: Правящая партия Организация народов Юго‑Западной Африки (СВАПО) также сохранила большинство на парламентских выборах.


Category: science/technology

Random text: Тигриный рык не похож на полнозвучный рык льва, а больше напоминает выкрикивание предложений из неразборчивых слов.


Category: sports

Random t

## Обернем тексты в prompt для llm

In [15]:
validation_set_for_llm = validation_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))
test_set_for_llm = test_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [16]:
print(validation_set_for_llm)

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})


In [17]:
print(validation_set['text'][0])

Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.


In [18]:
print(validation_set_for_llm['message_for_llm'][0])

[{'content': 'Вы - классификатор тем текстов.', 'role': 'system'}, {'content': 'Вы являетесь классификатором тем текстов. Вам будет предоставлен текст, и ваша задача — определить, какая тема из предложенного списка наиболее подходит этому тексту. В ответе укажите только название темы, никаких других комментариев не требуется.\n\nПредложенный список тем: entertainment, geography, health, politics, science/technology, sports и travel.\n\nНиже приведены примеры, чтобы вы могли понять, как работать с заданием:\nТекст: Метеорологическая служба Исландии также не сообщала о каких-либо признаках сейсмической активности в районе Геклы за прошедшие 48 часов.\nВаш ответ: geography\nТекст: Тигриный рык не похож на полнозвучный рык льва, а больше напоминает выкрикивание предложений из неразборчивых слов.\nВаш ответ: science/technology\nТекст: Правящая партия Организация народов Юго‑Западной Африки (СВАПО) также сохранила большинство на парламентских выборах.\nВаш ответ: politics\n\nТеперь ваш текст

## Сгенерируем ответы

In [19]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(validation_set_for_llm['message_for_llm'])
))
y_true = validation_set['category']

  0%|          | 0/99 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [20]:
print(y_pred[0])

[{'content': 'Вы - классификатор тем текстов.', 'role': 'system'}, {'content': 'Вы являетесь классификатором тем текстов. Вам будет предоставлен текст, и ваша задача — определить, какая тема из предложенного списка наиболее подходит этому тексту. В ответе укажите только название темы, никаких других комментариев не требуется.\n\nПредложенный список тем: entertainment, geography, health, politics, science/technology, sports и travel.\n\nНиже приведены примеры, чтобы вы могли понять, как работать с заданием:\nТекст: Метеорологическая служба Исландии также не сообщала о каких-либо признаках сейсмической активности в районе Геклы за прошедшие 48 часов.\nВаш ответ: geography\nТекст: Тигриный рык не похож на полнозвучный рык льва, а больше напоминает выкрикивание предложений из неразборчивых слов.\nВаш ответ: science/technology\nТекст: Правящая партия Организация народов Юго‑Западной Африки (СВАПО) также сохранила большинство на парламентских выборах.\nВаш ответ: politics\n\nТеперь ваш текст

In [21]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                    precision    recall  f1-score   support

           culture       0.00      0.00      0.00         0
     entertainment       0.67      0.67      0.67         9
         geography       1.00      0.88      0.93         8
            health       0.89      0.73      0.80        11
           history       0.00      0.00      0.00         0
             music       0.00      0.00      0.00         0
            nature       0.00      0.00      0.00         0
          politics       0.91      0.71      0.80        14
science/technology       0.88      0.84      0.86        25
          security       0.00      0.00      0.00         0
            sports       0.92      1.00      0.96        12
           traffic       0.00      0.00      0.00         0
         transport       0.00      0.00      0.00         0
            travel       0.88      0.75      0.81        20

          accuracy                           0.80        99
         macro avg       0.44      0.4

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


 ## Постобработка текста

In [22]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))

In [23]:
print(y_pred_with_normalization[0])

sports


In [24]:
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.67      0.67      0.67         9
         geography       1.00      0.88      0.93         8
            health       0.89      0.73      0.80        11
          politics       0.83      0.71      0.77        14
science/technology       0.88      0.84      0.86        25
            sports       0.67      1.00      0.80        12
            travel       0.80      0.80      0.80        20

          accuracy                           0.81        99
         macro avg       0.82      0.80      0.80        99
      weighted avg       0.82      0.81      0.81        99



## Тестовые данные

In [25]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(test_set_for_llm['message_for_llm'])
))
y_true = test_set['category']

  0%|          | 0/204 [00:00<?, ?it/s]

In [26]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                    precision    recall  f1-score   support

               art       0.00      0.00      0.00         0
     communication       0.00      0.00      0.00         0
     entertainment       0.93      0.68      0.79        19
         geography       0.78      0.82      0.80        17
            health       0.83      0.91      0.87        22
             music       0.00      0.00      0.00         0
          politics       0.94      0.97      0.95        30
science/technology       0.90      0.92      0.91        51
            sports       0.92      0.88      0.90        25
         transport       0.00      0.00      0.00         0
            travel       0.97      0.80      0.88        40
           weather       0.00      0.00      0.00         0

          accuracy                           0.87       204
         macro avg       0.52      0.50      0.51       204
      weighted avg       0.91      0.87      0.88       204



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Постобработка тестовых данных

In [27]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))

In [28]:
print(y_pred_with_normalization[0])

science/technology


In [29]:
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.93      0.68      0.79        19
         geography       0.78      0.82      0.80        17
            health       0.74      0.91      0.82        22
          politics       0.88      0.97      0.92        30
science/technology       0.90      0.92      0.91        51
            sports       0.81      0.88      0.85        25
            travel       0.97      0.80      0.88        40

          accuracy                           0.87       204
         macro avg       0.86      0.86      0.85       204
      weighted avg       0.88      0.87      0.87       204



Результат без изменений:
                    precision    recall  f1-score   support

     entertainment       0.82      0.74      0.78        19
         geography       0.64      0.94      0.76        17
            health       0.83      0.86      0.84        22
          politics       0.94      0.97      0.95        30
           science       0.88      0.90      0.89        51
            sports       0.78      0.84      0.81        25
            travel       0.97      0.70      0.81        40

          accuracy                           0.85       204
         macro avg       0.84      0.85      0.84       204
      weighted avg       0.86      0.85      0.85       204

Результат с другим системным промтом:

                    precision    recall  f1-score   support

     entertainment       0.75      0.63      0.69        19
         geography       0.75      0.88      0.81        17
            health       0.83      0.86      0.84        22
          politics       0.90      0.93      0.92        30
           science       0.86      0.96      0.91        51
            sports       0.78      0.84      0.81        25
            travel       0.97      0.72      0.83        40

          accuracy                           0.85       204
         macro avg       0.83      0.83      0.83       204
      weighted avg       0.85      0.85      0.85       204

Результат с другим системным промтом, обновил промт:

                    precision    recall  f1-score   support

     entertainment       0.88      0.74      0.80        19
         geography       0.87      0.76      0.81        17
            health       0.75      0.82      0.78        22
          politics       0.91      0.97      0.94        30
           science       0.86      0.96      0.91        51
            sports       0.79      0.88      0.83        25
            travel       0.97      0.78      0.86        40

          accuracy                           0.86       204
         macro avg       0.86      0.84      0.85       204
      weighted avg       0.87      0.86      0.86       204

Результат с другим системным промтом, обновил промт, добавил few shot promt:

                    precision    recall  f1-score   support

     entertainment       0.93      0.68      0.79        19
         geography       0.78      0.82      0.80        17
            health       0.74      0.91      0.82        22
          politics       0.88      0.97      0.92        30
           science       0.90      0.92      0.91        51
            sports       0.81      0.88      0.85        25
            travel       0.97      0.80      0.88        40

          accuracy                           0.87       204
         macro avg       0.86      0.86      0.85       204
      weighted avg       0.88      0.87      0.87       204
